# Understanding tf.Example w/ TF 2.0 

First, understand this is the world's worst named class - not an example - it is an Example of data (I guess).   Here I'm using TF 2.0beta - because the Understanding Images required it.    The on-line  tutorial shows you can use tf 1.14

https://www.tensorflow.org/beta/tutorials/load_data/images  
https://www.tensorflow.org/tutorials/load_data/tf_records

You will need these skills!  What is an image, tf.Example, serialized example etc.  
You won't get far with served models without this understanding.

## TensorFlow 2.0 Beta

### Do this THIRD, first, do UnderstandingTF_IO & UnderstandingImages.ipynb



In [ ]:
import os, pathlib
import random
import IPython.display as display

import numpy as np
import matplotlib.pyplot as plt

# !pip install -q tensorflow==2.0.0-beta1
# you'll get a tensorflow-serving-api error

import tensorflow as tf


In [ ]:
tf.__version__
AUTOTUNE = tf.data.experimental.AUTOTUNE
tf.enable_eager_execution()

## Globals

In [ ]:
PROJECT_DIR = os.getcwd()

IMAGE_DIR = os.path.join(PROJECT_DIR, "data/jpeg_images")
ANNOTATION_DIR = os.path.join(PROJECT_DIR, "data/annotations")

MODEL_PATH = os.path.join(PROJECT_DIR, "trained_model/export/Servo/1564778509")
LABEL_MAP = os.path.join(PROJECT_DIR, "code/cfa_prod_label_map.pbtxt")

# you can get data using the TrainModel_Step1_Local notebook
TEST_TFRECORDS_PATH =  os.path.join(PROJECT_DIR, "code/tfrecords/test/")
                                    


## Data

In [ ]:
image_glob = os.path.join(IMAGE_DIR, '*.jpg')
all_image_paths = tf.io.gfile.glob(image_glob)

# inplace shuffle
random.shuffle(all_image_paths)
all_image_paths[0]

## Helper Functions for data type conversions

### Note: 
To stay simple, this example only uses scalar inputs. The simplest way to handle non-scalar features is to use tf.serialize_tensor to convert tensors to binary-strings. Strings are scalars in tensorflow. Use tf.parse_tensor to convert the binary-string back to a tensor.

In [ ]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:
byte_string = b'test_string'
ft_byte_string = _bytes_feature(byte_string)
print("Feature - bytes:", type(ft_byte_string))
print("   value:", ft_byte_string)

# unencoded bytes - encoded to utf-8
print(_bytes_feature(u'test_bytes'.encode('utf-8')))

print(_float_feature(np.exp(1)))
print(_int64_feature(True))
print(_int64_feature(1))

### Non-Scalar Example - an image

In [ ]:
img_tensor = tf.io.read_file(all_image_paths[3])
print ("img tensor:", type(img_tensor), img_tensor.dtype)

# serialize the tensor
ser_img_tensor =tf.io.serialize_tensor(img_tensor)
print ("serialized:", type(ser_img_tensor), ser_img_tensor.dtype)

print (img_tensor == ser_img_tensor)

## Create tf.Example message

#### Make up some data

In [ ]:
# the number of observations in the dataset
n_observations = int(1e4)

# boolean feature, encoded as False or True
feature0 = np.random.choice([False, True], n_observations)
print (type(feature0), feature0.shape, feature0[:5])

# integer feature, random from 0 .. 4
feature1 = np.random.randint(0, 5, n_observations)
print (type(feature1), feature1.shape, feature1[:5])

# string feature - note: string as a byte array
strings = np.array([b'cat', b'dog', b'chicken', b'horse', b'goat'])
feature2 = strings[feature1]
print (type(feature2), feature2.shape, feature2[:5])

# float feature, from a standard normal distribution
feature3 = np.random.randn(n_observations)
print (type(feature3), feature3.shape, feature3[:5])

### Serialize Example

In [ ]:
def make_example(feature0, feature1, feature2, feature3):
  """
  Creates a tf.Example message ready to be written to a file.
  """

  # Create a dictionary mapping the feature name to the tf.Example-compatible
  # data type.

  feature = {
      'feature0': _int64_feature(feature0),
      'feature1': _int64_feature(feature1),
      'feature2': _bytes_feature(feature2),
      'feature3': _float_feature(feature3),
  }

  # Create a Features message using tf.train.Example.

  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto

def serialize_example(example_proto):
  return example_proto.SerializeToString()

In [ ]:
# This is an example observation from the dataset.

example_observation = []

example = make_example(False, 4, b'goat', 0.9876)
print ("example protobuf", type(example), '\n', example)

In [ ]:
example_serialized = serialize_example(example)
print ("example serialized", type(example_serialized), '\n', example_serialized)

In [ ]:
predict_dict = {'serialized_example' : example_serialized}
print (type(predict_dict), '\n', predict_dict)

In [ ]:
# research/utils/dataset_util/recurive_parse_xml_to_dict(xml)